In [8]:
import numpy as np
import pandas as pd
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from src.impact_modeling import ImpactModel

In [9]:
# 1. Load your enriched unified dataset
import pandas as pd
unified_path = "../data/processed/ethiopia_fi_unified_data.xlsx"

# Load observations/events and the impact links
df_main = pd.read_excel(unified_path, sheet_name='ethiopia_fi_unified_data')
df_links = pd.read_excel(unified_path, sheet_name='Impact_sheet')



In [10]:
df_links.head()

,record_id,parent_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,IMP_0001,EVT_0001,impact_link,NaN,ACCESS,Telebirr effect on Account Ownership,NaN,NaN,15.0,NaN,...,increase,high,15.0,12,literature,Kenya,Example_Trainee,2025-01-20,NaN,Kenya M-Pesa showed +20pp over 5 years
1,IMP_0002,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on Telebirr Users,NaN,NaN,NaN,NaN,...,increase,high,NaN,3,empirical,NaN,Example_Trainee,2025-01-20,NaN,Direct subscriber acquisition
2,IMP_0003,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on P2P Transactions,NaN,NaN,25.0,NaN,...,increase,high,25.0,6,empirical,NaN,Example_Trainee,2025-01-20,NaN,New digital payment channel
3,IMP_0004,EVT_0002,impact_link,NaN,ACCESS,Safaricom effect on 4G Coverage,NaN,NaN,15.0,NaN,...,increase,medium,15.0,12,empirical,NaN,Example_Trainee,2025-01-20,NaN,Network investment from competition
4,IMP_0005,EVT_0002,impact_link,NaN,AFFORDABILITY,Safaricom effect on Data Affordability,NaN,NaN,-20.0,NaN,...,decrease,medium,-20.0,12,literature,Rwanda,Example_Trainee,2025-01-20,NaN,Competition typically reduces prices


# Building the Event–Indicator Matrix (Impact Link Modeling)

This section describes how we translated the structured `impact_link` relationships into an **Event–Indicator Matrix** for forecasting financial inclusion indicators in Ethiopia.

Each `impact_link` record captures:

- **Event:** Identified by `parent_id`  
- **Affected indicator:** `related_indicator`  
- **Direction of impact:** `impact_direction`  
- **Magnitude or estimate:** Either numeric (`impact_estimate`) or qualitative (`impact_magnitude`)  
- **Lag effect:** `lag_months`, if applicable  

The goal is to convert these relationships into a **quantitative model** that predicts indicator changes when events occur.

---

## 1. Representing Event Effects Quantitatively

To operationalize the impact of each event, we calculate a numeric **weight** for every event–indicator pair.  

### Weight Calculation

Within the `ImpactModel` class, weights are determined as follows:

1. **Primary source:** `impact_estimate`  
   - If a numeric value is provided, it is used directly.

2. **Fallback:** Qualitative magnitude mapping  
   - If `impact_estimate` is missing or zero, we map qualitative labels to numeric values:

| Qualitative Magnitude | Numeric Value |
|----------------------|--------------|
| High                 | 0.8          |
| Medium               | 0.5          |
| Low                  | 0.2          |
| Default fallback     | 0.1          |

This ensures every link contributes a usable weight, even when precise estimates are unavailable.

---

## 2. Incorporating Impact Direction

Event impacts can be **positive** or **negative**.  

The model applies a **direction multiplier**:

- **Increase / Positive → +1**  
- **Decrease / Negative → −1**  

The final weight for each link is therefore:

\[
\text{weight} = \text{magnitude} \times \text{direction multiplier} \times \text{evidence weight}
\]

**Example:**

- Telebirr launch increases mobile money accounts → weight: `+4.75`  
- Macroeconomic shock reduces affordability → weight: `-0.5`  

This step ensures the matrix reflects both the **strength** and **sign** of each event’s effect.

---

## 3. Constructing the Event–Indicator Matrix

Once weights are computed, the data is **pivoted** into a matrix:

- **Rows:** Events (`parent_id`)  
- **Columns:** Indicators (`related_indicator`)  
- **Values:** Computed event impact weights  

This matrix forms the core input for forecasting indicator trajectories and simulating multi-event effects over time.


In [11]:
model = ImpactModel(df_main, df_links)

# 1. Build matrix
matrix = model.generate_matrix()
display(matrix)

# 2. Forecast indicator
print("2. Forecast indicator")
forecast_mm = model.simulate_indicator("ACC_MM_ACCOUNT")

# 3. Validate Telebirr vs reality
print("3. Validate Telebirr vs reality")
validation = model.validate_event("ACC_MM_ACCOUNT", 2021, 2024)


print(validation)

# 4. Print methodology
print("4. Print methodology")
print(model.methodology_text())

output_file = "../data/processed/event_association_matrix.xlsx"
matrix.to_excel(output_file, index=True)


related_indicator,ACC_4G_COV,ACC_FAYDA_TOTAL,ACC_MM_ACCOUNT,ACC_OWNERSHIP,AFF_DATA_INCOME,GEN_GAP_ACC,GEN_GAP_OWNERSHIP,MACRO_FX_RATE,MACRO_INFLATION,USG_FUEL_DIG_VAL,USG_MPESA_ACTIVE,USG_MPESA_USERS,USG_P2P_COUNT,USG_P2P_INTEROP_COUNT,USG_TELEBIRR_REG,USG_TELEBIRR_USERS
parent_id,,,,,,,,,,,,,,,,
EVT_0001,0.0,0.00,0.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.8
EVT_0002,15.0,0.00,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EVT_0003,0.0,0.00,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0
EVT_0004,0.0,0.00,0.0,7.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EVT_0005,0.0,0.00,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EVT_0007,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.5,0.0,7.0,0.0,0.0,0.0
EVT_0008,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0
EVT_0010,0.0,0.00,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EVT_0011,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0


2. Forecast indicator


,date,baseline,EVT_0003,total_event_effect,predicted
0,2021-01-31,4.7,0.000000,0.000000,4.700000
1,2021-02-28,4.7,0.000000,0.000000,4.700000
2,2021-03-31,4.7,0.000000,0.000000,4.700000
3,2021-04-30,4.7,0.000000,0.000000,4.700000
4,2021-05-31,4.7,0.000000,0.000000,4.700000
...,...,...,...,...,...
79,2027-08-31,4.7,1.999996,1.999996,6.699996
80,2027-09-30,4.7,1.999997,1.999997,6.699997
81,2027-10-31,4.7,1.999998,1.999998,6.699998
82,2027-11-30,4.7,1.999998,1.999998,6.699998


3. Validate Telebirr vs reality


,Indicator,Observed Change,Predicted Change,Difference
0,ACC_MM_ACCOUNT,4.75,1.931905,-2.818095


        Indicator  Observed Change  Predicted Change  Difference
0  ACC_MM_ACCOUNT             4.75          1.931905   -2.818095
4. Print methodology

# Event Impact Modeling Methodology

## Overview
Each event (policy, product launch, infrastructure reform) is modeled as a shock
that affects one or more financial inclusion indicators.

## Impact Links
Each impact link provides:
- Indicator affected
- Direction (increase/decrease)
- Magnitude estimate
- Lag in months
- Evidence basis (empirical/literature/theoretical)

## Time Dynamics
Impacts do not occur instantly.  
We apply a gradual adoption curve:

Effect(t) = Impact × (1 − exp(−k(t − lag)))

## Combining Multiple Events
Indicator trajectories are additive:

Predicted(t) = Baseline(t) + Σ EventEffects(t)

## Comparable Country Evidence
When Ethiopia-specific pre/post data is limited:
- Literature evidence from Kenya, India, Tanzania is used
- These impacts are down-weighted relative to empirical estimates

## Validation
Telebirr

# Event–Indicator Impact Summary (Association Matrix Interpretation)

This section summarizes the modeled relationships between national-level events and financial inclusion indicators in Ethiopia.  
Each event affects one or more indicators with an estimated impact magnitude, representing the strength and direction of influence.

---

## EVT_0001 — P2P Regulation Update

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| ACC_OWNERSHIP     | +15             |
| USG_P2P_COUNT     | +25             |
| USG_TELEBIRR_USERS| +0.1            |

**Interpretation:** Regulatory changes strongly increase digital transfer usage and improve overall account ownership.

---

## EVT_0002 — Infrastructure Expansion / Network Upgrade

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| ACC_4G_COV        | +15             |
| AFF_DATA_INCOME   | +20             |

**Interpretation:** Improved connectivity and affordability enable broader digital financial access.

---

## EVT_0003 — Mobile Money Growth Trigger

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| ACC_MM_ACCOUNT    | +5              |
| USG_MPESA_USERS   | +0.1            |

**Interpretation:** This event supports growth in mobile money account penetration and early M-Pesa adoption.

---

## EVT_0004 — Financial Access Policy Reform

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| ACC_OWNERSHIP     | +10             |
| GEN_GAP_ACC       | +5              |

**Interpretation:** Policy reform increases account ownership and reduces inequality in access.

---

## EVT_0005 — Data & Digital Cost Shock

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| AFF_DATA_INCOME   | +30             |

**Interpretation:** This event has the strongest modeled impact on digital affordability conditions.

---

## EVT_0007 — M-Pesa Market Entry & Activation Push

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| USG_FUEL_DIG_VAL  | +15             |
| USG_P2P_COUNT     | +10             |

**Interpretation:** M-Pesa expansion increases digital transaction activity and P2P usage.

---

## EVT_0008 — P2P Adoption Milestone

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| USG_P2P_COUNT     | +15             |

**Interpretation:** A milestone event accelerating peer-to-peer digital payment adoption.

---

## EVT_0010 — Affordability Policy Intervention

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| AFF_DATA_INCOME   | +10             |

**Interpretation:** Moderate improvement in affordability supporting inclusion growth.

---

## EVT_0011 — Telebirr Product Launch

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| USG_TELEBIRR_REG  | +50             |

**Interpretation:** Telebirr launch is the largest direct driver of mobile money registrations.

---

## EVT_0012 — Macroeconomic Shift

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| MACRO_FX_RATE     | +124.5          |
| MACRO_INFLATION   | −9.4            |

**Interpretation:** Economic instability influences exchange rate pressures and inflation dynamics.

---

## EVT_0013 — Digital Value Surge

| Indicator Affected | Impact Magnitude |
|-------------------|-----------------|
| USG_FUEL_D
